In [7]:
import pandas as pd
import numpy as np 
import json

In [8]:
data = pd.read_json('data/manifest.jsonl', lines=True)

data

,id,audio_filepath,text,duration
0,e632f7d39c15e7edfc665b91e6f2071f,files/e632f7d39c15e7edfc665b91e6f2071f.wav,афина воспроизведи музыку вперемешку,4.900000
1,5db5df8bb9e3b6660b2a04b34d4a355d,files/5db5df8bb9e3b6660b2a04b34d4a355d.wav,найти сериал григорий р,3.652000
2,2c471aedc6979109f28cd53c58f8c4fb,files/2c471aedc6979109f28cd53c58f8c4fb.wav,прямой эфир апл манчестер юнайтед тоттенхэм,4.341750
3,756a137ee9debde4a008adc4a4121dc7,files/756a137ee9debde4a008adc4a4121dc7.wav,ильвиром ивановичем ворончихиным,3.900000
4,1ee3b00170123a6723a40e129b2f6bce,files/1ee3b00170123a6723a40e129b2f6bce.wav,можешь показать киношку исходный код,3.320000
...,...,...,...,...
9989,73bb9272fe3724eb6212e33186f63152,files/73bb9272fe3724eb6212e33186f63152.wav,рудню,2.260000
9990,c872c6f50709a2696b0927725dc86ded,files/c872c6f50709a2696b0927725dc86ded.wav,найти мульт щенячий патруль,3.020000
9991,03b92263995462b2e09655a8e29d4d0b,files/03b92263995462b2e09655a8e29d4d0b.wav,стол из эпоксидной смолы,3.598688
9992,09ac1f9718c6da18380b4a01d92d2561,files/09ac1f9718c6da18380b4a01d92d2561.wav,шесть четыреста семьдесят шесть четыреста двад...,11.795312


In [9]:
i = np.random.randint(0, data.shape[0])

audio_id = data['id'].iloc[i]
audio_path = data['audio_filepath'].iloc[i]
audio_text = data['text'].iloc[i]

def compare_audio(result, real, similarity):
    print(f'Recognized is "{str(result).lower()}",',  
          f'\nwhile real is "{str(real).lower()}".',  
          f'\nSimilarity is {similarity}')
    
def dump_text(result, audio_id):
    with open(f'data/texts/{audio_id}'+'_text.txt', 'w') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

Speech Recognition (google)

In [10]:
import speech_recognition as sr 

In [11]:
recog = sr.Recognizer()

with sr.AudioFile('data/'+audio_path) as audio:
    rec = recog.record(audio)
    
result = recog.recognize_google(audio_data=rec, language='ru-RU') 

similarity = str(result).lower() == str(audio_text).lower()

compare_audio(result, audio_text, similarity)
dump_text(result, audio_id)

Recognized is "к какому году относятся первые русские сообщения о байкале", 
while real is "к какому году относятся первые русские сообщения о байкале". 
Similarity is True


Vosk

In [12]:
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment

In [ ]:
model = Model("model")
recog = KaldiRecognizer(model, 16000)
recog.SetWords(True)

# Используя библиотеку pydub делаем предобработку аудио
wav = (AudioSegment.from_wav(audio_path)
       .set_channels(1)
       .set_frame_rate(16000))

# Преобразуем вывод в json
recog.AcceptWaveform(wav.raw_data)
result = recog.Result()
text = json.loads(result)["text"]

# Записываем результат в файл "data.txt"
with open('data.txt', 'w') as f:
    json.dump(text, f, ensure_ascii=False, indent=4)